# KPI Example

In this example we look at the options for KPI tracking and their evaluation.

To illustrate the default settings, we first create a simulation environment, using the simple data center model.

In [1]:
import energym

env = energym.make("DatacenterThermostat-v0", simulation_days=300)

[OK] The current working directory is %s

[OK] fmiInstantiateSlave: Path to fmuUnzipLocation is not %s.

[OK] fmiInstantiateSlave: Path to fmuUnzipLocation is not %s.

[OK] fmiInstantiateSlave: Path to fmuUnzipLocation is not %s.

[OK] Command executes to copy content of resources folder: %s

[OK] fmiInstantiateSlave: Path to model description file is %s.

[OK] fmiInstantiateSlave: The FMU modelIdentifier is %s.

[OK] fmiInstantiateSlave: The FMU modelGUID is %s.

[OK] fmiInstantiateSlave: Slave %s is instantiated.

[OK] fmiInstantiateSlave: Instantiation of %s succeded.



During the simulation, the important values for the KPIs are stored in a KPI object and can be retrieved with the `get_kpi()` method.

In [2]:
for _ in range(500):
    action = env.sample_random_action()
    output = env.step(action)

print(env.get_kpi())

[OK] fmiInitializeSlave: The sockfd is %d.

[OK] fmiInitializeSlave: The port number is %d.

[OK] fmiInitializeSlave: This hostname is %s.

[OK] fmiInitializeSlave: TCPServer Server waiting for clients on port: %d.

[OK] fmiInitializeSlave: The number of input variables is %d.

[OK] fmiInitializeSlave: The number of output variables is %d.

[OK] Get input file from resource folder %s.

[OK] Searching for following pattern %s

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Read directory and search for *.idf, *.epw, or *.idd file.

[OK] Found matching file %s.

[OK] done searching pattern %s

[OK] Get input file from resource folder %s.

[OK] Se

Through this, we can directly see the default KPIs that are tracked for the data center model. Those are
* The average power demand of the whole facility
* The average temperature deviation from an interval for both zones
* The total temperature interval violations for both zones

In the example above, we did not specify any arguments for `get_kpi()`, so the KPIs were computed over all performed simulation steps. To get the KPIs from a specific time interval, we can specify the simulation steps in the method call:

In [3]:
print(env.get_kpi(start_ind=100, end_ind=300))

{'kpi1': {'name': 'Fa_Pw_All', 'type': 'avg', 'kpi': 76609.32401534989}, 'kpi2': {'name': 'Z01_T', 'type': 'avg_dev', 'target': [17, 26], 'kpi': 0.07871807946519971}, 'kpi3': {'name': 'Z02_T', 'type': 'avg_dev', 'target': [17, 26], 'kpi': 0.02236341982219245}, 'kpi4': {'name': 'Z01_T', 'type': 'tot_viol', 'target': [17, 26], 'kpi': 26}, 'kpi5': {'name': 'Z02_T', 'type': 'tot_viol', 'target': [17, 26], 'kpi': 10}}


The `get_kpi()` method comes in handy if we want to use the KPI values somewhere in the code. To simply print the KPIs, we can directly use the `print_kpis()` method:

In [4]:
env.print_kpis()

####################################################################
Variable name: Fa_Pw_All, kpi type: avg, kpi value: 77271.67873206396
####################################################################
Variable name: Z01_T, kpi type: avg_dev, kpi value: 0.08053484806975367
####################################################################
Variable name: Z02_T, kpi type: avg_dev, kpi value: 0.021179971411713086
####################################################################
Variable name: Z01_T, kpi type: tot_viol, kpi value: 62
####################################################################
Variable name: Z02_T, kpi type: tot_viol, kpi value: 26


Sometimes we want to evaluate KPI for more than one zone, for example the average temperature deviation or the number of temperature violations over all zones. For this, the `get_cumulative_kpi()` method can be used. We need to specify a part of the name that all relevant KPIs have in common (here e.g. "_T") the KPI type that should be aggregated (e.g. "avg_dev") and the type of aggregation (e.g. "avg"). In code this looks as follows:

In [5]:
print(env.get_cumulative_kpi(phrase="_T", kpi_type="avg_dev", out_type="avg"))
print(env.get_cumulative_kpi(phrase="_T", kpi_type="tot_viol", out_type="sum"))

env.close()

0.05085740974073338
88
[OK] fmiTerminateSlave: fmiFreeInstanceSlave must be called to free the FMU instance.

[OK] fmiFreeSlaveInstance: The function fmiFreeSlaveInstance of instance %s is executed.

[OK] freeInstanceResources: %s will be freed.



## Specifying custom KPIs

To track custom KPIs instead of default ones, it is possible to specify the kpi_options parameter in the construction of the simulation environment. We will illustrate this through tracking the average power demand at the HVAC, building and facility level, the total facility power demand and the average deviation of the zone temperatures from a fixed target:

In [6]:
kpi_options = {"kpi1": {"name":"Fa_Pw_All", "type":"avg"},
              "kpi2": {"name":"Bd_Pw_All", "type":"avg"},
              "kpi3": {"name":"Fa_Pw_HVAC", "type":"avg"},
              "kpi4": {"name":"Fa_Pw_All", "type":"sum"},
              "kpi5": {"name":"Z01_T", "type":"avg_dev", "target":23},
              "kpi6": {"name":"Z02_T", "type":"avg_dev", "target":23}
              }

env = energym.make("DatacenterThermostat-v0", simulation_days=300, kpi_options=kpi_options)

[OK] The current working directory is %s

[OK] fmiInstantiateSlave: Path to fmuUnzipLocation is not %s.

[OK] fmiInstantiateSlave: Path to fmuUnzipLocation is not %s.

[OK] fmiInstantiateSlave: Path to fmuUnzipLocation is not %s.

[OK] Command executes to copy content of resources folder: %s

[OK] fmiInstantiateSlave: Path to model description file is %s.

[OK] fmiInstantiateSlave: The FMU modelIdentifier is %s.

[OK] fmiInstantiateSlave: The FMU modelGUID is %s.

[OK] fmiInstantiateSlave: Slave %s is instantiated.

[OK] fmiInstantiateSlave: Instantiation of %s succeded.



We run this simulation again for a few timesteps with random actions and observe the KPIs:

In [8]:
for _ in range(500):
    action = env.sample_random_action()
    output = env.step(action)

env.print_kpis()

env.close()

####################################################################
Variable name: Fa_Pw_All, kpi type: avg, kpi value: 80488.52614499704
####################################################################
Variable name: Bd_Pw_All, kpi type: avg, kpi value: 72127.17908818509
####################################################################
Variable name: Fa_Pw_HVAC, kpi type: avg, kpi value: 8361.347056811981
####################################################################
Variable name: Fa_Pw_All, kpi type: sum, kpi value: 80488526.14499705
####################################################################
Variable name: Z01_T, kpi type: avg_dev, kpi value: 3.267533214577538
####################################################################
Variable name: Z02_T, kpi type: avg_dev, kpi value: 2.976782357648463
[OK] fmiTerminateSlave: fmiFreeInstanceSlave must be called to free the FMU instance.

[OK] fmiFreeSlaveInstance: The function fmiFreeSlaveInstance of instance %s is